# 1/ Generate ASPIC-arguments


## 1.1/ Basic classes


### 1.1.1/ Literal

In [15]:
class Literal:
    def __init__(self, name, is_negativ=False):
        assert isinstance(name, str), "name should be a string"
        assert isinstance(is_negativ, bool), "is_negativ should be a booleean"
        self.name = name
        self.is_negativ = is_negativ

    def __repr__(self):
        if self.is_negativ:
            return f"!{self.name}"
        else:
            return self.name

    def __eq__(self, other):
        return (
            isinstance(other, Literal) and
            self.name == other.name and
            self.is_negativ == other.is_negativ
        )

    def __hash__(self):
        return hash((self.name, self.is_negativ))

    def negate(self):
        self.is_negativ = not self.is_negativ

    def copy(self):
        return Literal(self.name, self.is_negativ)

# Example usage:
literal1 = Literal("a", True) # Expect !a
literal2 = Literal("b") # Expect b
literal3 = Literal("c", True) # Expect !c

print("L1:", literal1)
print("L2:", literal2)
print("L3:", literal3)

literal1.negate() #Expect a
print("L1 nagative :", literal1)

print("L1 == L2:", literal1 == literal2)  # Expect : False
print("L1 == L3:", literal1 == literal3)  # Expect : True


L1: !a
L2: b
L3: !c
L1 nagative : a
L1 == L2: False
L1 == L3: False


### 1.1.2/ Rule

In [16]:
class Rule:
    def __init__(self,premises, conclusion, is_defeasible=False, reference=None, pref = 0):
        assert isinstance(premises, list), "premises should be a list"
        assert all([isinstance(p, Literal) for p in premises]), "premises should be a list of Literal"
        assert isinstance(conclusion, Literal), "conclusion should be a Literal"
        assert isinstance(is_defeasible, bool), "is_defeasible should be a boolean"
        assert reference is None or isinstance(reference, Literal), "reference must be a Literal or None"
        assert isinstance(pref,int), " pref should be a integer"
        
        self.premises = premises
        self.conclusion = conclusion
        self.is_defeasible = is_defeasible
        self.reference = reference
        self.pref = pref

    def __repr__(self):
            premises_str = ", ".join(map(str, self.premises))
            conclusion_str = str(self.conclusion)
            reference_str = f"[{self.reference}]" if self.reference is not None else "None"
            pref_str = str(self.pref)
            if self.is_defeasible:
                return f"{reference_str}: {premises_str} => {conclusion_str} ({pref_str}) "
            else:
                self.pref = True
                return f"{reference_str}: {premises_str} -> {conclusion_str} "

    def __eq__(self, other):
        return (
            isinstance(other, Rule) and
            self.premises == other.premises and
            self.conclusion == other.conclusion and
            self.is_defeasible == other.is_defeasible and
            self.reference == other.reference
        )

    def __hash__(self):
        return hash((tuple(self.premises), self.conclusion, self.is_defeasible, self.reference))

# Example usage:
premises = [Literal("a"), Literal("b", is_negativ=True)]
conclusion = Literal("c")
r1 = Rule(premises, conclusion, is_defeasible=True, reference=Literal("r1"), pref = 0)
r2 = Rule(premises, conclusion, is_defeasible=False, reference=Literal("r2"), pref = 1)
r3 = Rule(premises, conclusion, is_defeasible=True, reference=Literal("r1"))

print(r1) #Expect [r1]: a, !b => c (0)
print(r2) #Expect [r2]: a, !b -> c 

print("r1 == r2:", r1 == r2)  # Expect : False
print("r1 == r3:", r1 == r3)  # Expect : True

[r1]: a, !b => c (0) 
[r2]: a, !b -> c 
r1 == r2: False
r1 == r3: True


### 1.1.3/ Argument

In [17]:
class Argument:
    def __init__(self, top_rule, sub_arguments, name=None):
        self.top_rule = top_rule
        self.sub_arguments = sub_arguments if sub_arguments else []
        self.name = name

    def __str__(self):
        arrow = "->" if not self.top_rule.is_defeasible else "=>"
        if not self.sub_arguments:
            return f"{self.name} : {arrow} {self.top_rule.conclusion}"
        else:
            sub_args_str = ", ".join(str(arg.name) for arg in self.sub_arguments)
            return f"{self.name} : {sub_args_str} {arrow} {self.top_rule.conclusion}"


    def __eq__(self, other):
        self_sub_arg_names = set(arg.name for arg in self.sub_arguments)
        other_sub_arg_names = set(arg.name for arg in other.sub_arguments)
        return (
            isinstance(other, Argument) and
            self.top_rule == other.top_rule and
            self_sub_arg_names == other_sub_arg_names and
            self.name == other.name
        )

    def __hash__(self):
        return hash((self.top_rule, tuple(self.sub_arguments), self.name))


    def all_defeasible_rules(self):
        defeasible_rules = set()
        if self.top_rule.is_defeasible:
            defeasible_rules.add(self.top_rule)
        for sub_arg in self.sub_arguments:
            if sub_arg.top_rule.is_defeasible:
                defeasible_rules.add(sub_arg.top_rule)
            defeasible_rules.update(sub_arg.all_defeasible_rules())
        return defeasible_rules
    
    def last_defeasible_rule(self):
        defeasible_rules = set()
        if self.top_rule.is_defeasible:
            defeasible_rules.add(self.top_rule)
        else : 
            for sub_arg in self.sub_arguments:
                defeasible_rules.update(sub_arg.last_defeasible_rule()) 
        return defeasible_rules

    
    def sub_arguments_set(self):
        sub_args = set()
        for arg in self.sub_arguments:
            sub_args.add(arg)
            sub_args.update(arg.sub_arguments_set())  # Recursively collect sub-arguments
        return sub_args

    def compare(self, other_arg, preferences, principle):

        self_rule = self.top_rule
        other_rule = other_arg.top_rule

        
        if self_rule in preferences and other_rule in preferences[self_rule]:
            return 1
        elif other_rule in preferences and self_rule in preferences[other_rule]:
            return -1
        else:
            if principle == 'democratic':
                return len(self.sub_arguments) - len(other_arg.sub_arguments)
            elif principle == 'elitist':
                return len([arg for arg in self.sub_arguments if not arg.sub_arguments]) - len([arg for arg in other_arg.sub_arguments if not arg.sub_arguments])
            elif principle == 'last-link':
                return len(self.last_defeasible_rules()) - len(other_arg.last_defeasible_rules())
            elif principle == 'weakest-link':
                return len(self.all_defeasible_rules()) - len(other_arg.all_defeasible_rules())
            else:
                raise ValueError("Invalid principle specified.")
            
# Example usage
rx = Rule([], Literal("a"), False, reference=Literal("rx"))
ry = Rule([Literal("a")], Literal("c"), True, reference=Literal("ry"))
rz = Rule([Literal("c"), Literal("a")], Literal("d"), False, reference=Literal("rz"))
arg1 = Argument(rx, [], 'A1')
arg2 = Argument(ry, [arg1], 'A2')
arg4 = Argument(rz, [arg2,arg1], 'A3')
arg3 = Argument(r1, [], 'A1')

print(arg1)
print(arg2)
print(arg4)
print("arg1 == arg2:", arg1 == arg2)  # Expect : False
print("arg1 == arg3:", arg1 == arg3)  # Expect : True

A1 : -> a
A2 : A1 => c
A3 : A2, A1 -> d
arg1 == arg2: False
arg1 == arg3: False


### 1.1.4/ Contraposition

In [18]:
it=0
def contraposition(rule):
    if rule.is_defeasible:
        raise ValueError("Contraposition rules can only be created for strict rules.")
    global it
    premises = rule.premises
    conclusion = rule.conclusion
    contrapositives = []
    for premise in premises:
        conclusion_neg = conclusion.copy()
        conclusion_neg.negate()

        premise_neg = premise.copy()
        premise_neg.negate()

        premise_temp = [conclusion_neg]

        for prem in premises:
          if prem != premise:
            premise_temp.append(prem)
        contra_name = "c" + str(it)
        contrapositives.append(Rule(premise_temp, premise_neg , is_defeasible=False, reference = Literal(contra_name)))
        it = it +1

    return contrapositives

def affiche_contra (contra):
    for contrapositive in contra:
        print(contrapositive)

# Example usage:
premises = [Literal("b"), Literal("d"), Literal("e", True)]
conclusion = Literal("c")
strict_rule = Rule(premises, conclusion, is_defeasible=False, reference=Literal("r3"))

print("Règle initiale:")
print(strict_rule)
print("Contraposition:")
contra = contraposition(strict_rule)
affiche_contra(contra)

Règle initiale:
[r3]: b, d, !e -> c 
Contraposition:
[c0]: !c, d, !e -> !b 
[c1]: !c, b, !e -> !d 
[c2]: !c, b, d -> e 


### 1.1.5/ All Rules

In [19]:
# Strict rule
r1 = Rule([], Literal("a"),False, reference=Literal("r1"))
r3 = Rule([Literal("d"), Literal("b")], Literal("c"), False, reference=Literal("r3"))
r5 = Rule([Literal("c", True)], Literal("d"), False, reference=Literal("r5"))

# Defeasible rule
r2 = Rule([Literal("a")], Literal("d", True),True, reference=Literal("r2"))
r4 = Rule([], Literal("b"),True, reference=Literal("r4"),  pref = True)
r6 = Rule([], Literal("c", True),True, reference=Literal("r6"), pref = True)
r7 = Rule([], Literal("d"),True, reference=Literal("r7"))
r8 = Rule([Literal("c")], Literal("e"),True, reference=Literal("r8"))
r9 = Rule([Literal("c", True)], Literal("r2", True),True, reference=Literal("r9"))

r10 = contraposition(r1)
r11 = contraposition(r3)
r12 = contraposition(r5)

ListOfRules = []
ListOfRules = [r1, r2, r3, r4, r5, r6, r7, r8, r9]
ListOfRules.extend(r10)
ListOfRules.extend(r11)
ListOfRules.extend(r12)

# Affichage de tous les éléments de ListOfArgument
for rule in ListOfRules:
    print(rule)

[r1]:  -> a 
[r2]: a => !d (0) 
[r3]: d, b -> c 
[r4]:  => b (True) 
[r5]: !c -> d 
[r6]:  => !c (True) 
[r7]:  => d (0) 
[r8]: c => e (0) 
[r9]: !c => !r2 (0) 
[c3]: !c, b -> !d 
[c4]: !c, d -> !b 
[c5]: !d -> c 


## 1.2/ Generating arguments

In [20]:
Argnamect = 1

def afficher_arg(arguments):
    for argument in arguments:
        print(argument)

def same_arg(arguments, new_arg):
    same = False
    for arg in arguments:
        if new_arg.top_rule == arg.top_rule and new_arg.sub_arguments == arg.sub_arguments:
            same = True
    return same

from itertools import combinations

def generate_argument_combinations(rule, arguments):
    all_combinations = []
    for r in range(1, len(rule.premises) + 1):  # Parcourir toutes les tailles possibles de combinaison
        for comb in combinations(arguments, r):  # Générer toutes les combinaisons possibles de taille r
            if all(premise in [arg.top_rule.conclusion for arg in comb] for premise in rule.premises):
                all_combinations.append(comb)  # Ajouter la combinaison si elle satisfait toutes les prémisses
    return all_combinations

def generate_arguments(rules):
    arguments = []
    ListRules = list(rules)
    global Argnamect

    for rule in rules:
        if not rule.premises:
            name = "A" + str(Argnamect)
            arguments.append(Argument(rule, [], name))
            Argnamect += 1
            ListRules.remove(rule)

    new_arguments_generated = True

    while new_arguments_generated:
        new_arguments_generated = False
        for lr in ListRules:
            if lr.premises:
                all_combinations = generate_argument_combinations(lr, arguments)
                for combination in all_combinations:
                    new_arg = Argument(lr, combination, "A" + str(Argnamect))
                    if not same_arg(arguments, new_arg):
                        arguments.append(new_arg)
                        Argnamect += 1
                        new_arguments_generated = True
                        break
                if new_arguments_generated:
                    break

    return arguments

arguments = generate_arguments(ListOfRules)

print("\nAffichage des arguments")
afficher_arg(arguments)


Affichage des arguments
A1 : -> a
A2 : => b
A3 : => !c
A4 : => d
A5 : A1 => !d
A6 : A2, A4 -> c
A7 : A3 -> d
A8 : A2, A7 -> c
A9 : A6 => e
A10 : A8 => e
A11 : A3 => !r2
A12 : A2, A3 -> !d
A13 : A3, A4 -> !b
A14 : A3, A7 -> !b
A15 : A5 -> c
A16 : A15 => e
A17 : A12 -> c
A18 : A17 => e


In [21]:
for arg in arguments:
    print ( " ******** ")
    print(arg)
    print ( "------ ") 
    def_rul = arg.all_defeasible_rules()
    print(def_rul)
    print ( " ******** ")

 ******** 
A1 : -> a
------ 
set()
 ******** 
 ******** 
A2 : => b
------ 
{[r4]:  => b (True) }
 ******** 
 ******** 
A3 : => !c
------ 
{[r6]:  => !c (True) }
 ******** 
 ******** 
A4 : => d
------ 
{[r7]:  => d (0) }
 ******** 
 ******** 
A5 : A1 => !d
------ 
{[r2]: a => !d (0) }
 ******** 
 ******** 
A6 : A2, A4 -> c
------ 
{[r4]:  => b (True) , [r7]:  => d (0) }
 ******** 
 ******** 
A7 : A3 -> d
------ 
{[r6]:  => !c (True) }
 ******** 
 ******** 
A8 : A2, A7 -> c
------ 
{[r4]:  => b (True) , [r6]:  => !c (True) }
 ******** 
 ******** 
A9 : A6 => e
------ 
{[r4]:  => b (True) , [r8]: c => e (0) , [r7]:  => d (0) }
 ******** 
 ******** 
A10 : A8 => e
------ 
{[r4]:  => b (True) , [r8]: c => e (0) , [r6]:  => !c (True) }
 ******** 
 ******** 
A11 : A3 => !r2
------ 
{[r9]: !c => !r2 (0) , [r6]:  => !c (True) }
 ******** 
 ******** 
A12 : A2, A3 -> !d
------ 
{[r4]:  => b (True) , [r6]:  => !c (True) }
 ******** 
 ******** 
A13 : A3, A4 -> !b
------ 
{[r7]:  => d (0) , [r6]:  => 

In [22]:
for arg in arguments:
    print(" ******** ")
    print(arg)
    print("------ ")
    def_rul_last = arg.last_defeasible_rule()
    print("Last Defeasible Rule:", def_rul_last)
    print(" ******** ")

 ******** 
A1 : -> a
------ 
Last Defeasible Rule: set()
 ******** 
 ******** 
A2 : => b
------ 
Last Defeasible Rule: {[r4]:  => b (True) }
 ******** 
 ******** 
A3 : => !c
------ 
Last Defeasible Rule: {[r6]:  => !c (True) }
 ******** 
 ******** 
A4 : => d
------ 
Last Defeasible Rule: {[r7]:  => d (0) }
 ******** 
 ******** 
A5 : A1 => !d
------ 
Last Defeasible Rule: {[r2]: a => !d (0) }
 ******** 
 ******** 
A6 : A2, A4 -> c
------ 
Last Defeasible Rule: {[r4]:  => b (True) , [r7]:  => d (0) }
 ******** 
 ******** 
A7 : A3 -> d
------ 
Last Defeasible Rule: {[r6]:  => !c (True) }
 ******** 
 ******** 
A8 : A2, A7 -> c
------ 
Last Defeasible Rule: {[r4]:  => b (True) , [r6]:  => !c (True) }
 ******** 
 ******** 
A9 : A6 => e
------ 
Last Defeasible Rule: {[r8]: c => e (0) }
 ******** 
 ******** 
A10 : A8 => e
------ 
Last Defeasible Rule: {[r8]: c => e (0) }
 ******** 
 ******** 
A11 : A3 => !r2
------ 
Last Defeasible Rule: {[r9]: !c => !r2 (0) }
 ******** 
 ******** 
A12 : A2, A

In [23]:
for i in range(18):
    print(" ***** ")
    print(arguments[i])
    print(" --- ")
    def_rul = arguments[i].sub_arguments_set()
    for sub in def_rul:
        print(sub)
    print(" ***** ")


 ***** 
A1 : -> a
 --- 
 ***** 
 ***** 
A2 : => b
 --- 
 ***** 
 ***** 
A3 : => !c
 --- 
 ***** 
 ***** 
A4 : => d
 --- 
 ***** 
 ***** 
A5 : A1 => !d
 --- 
A1 : -> a
 ***** 
 ***** 
A6 : A2, A4 -> c
 --- 
A4 : => d
A2 : => b
 ***** 
 ***** 
A7 : A3 -> d
 --- 
A3 : => !c
 ***** 
 ***** 
A8 : A2, A7 -> c
 --- 
A7 : A3 -> d
A2 : => b
A3 : => !c
 ***** 
 ***** 
A9 : A6 => e
 --- 
A4 : => d
A2 : => b
A6 : A2, A4 -> c
 ***** 
 ***** 
A10 : A8 => e
 --- 
A7 : A3 -> d
A2 : => b
A8 : A2, A7 -> c
A3 : => !c
 ***** 
 ***** 
A11 : A3 => !r2
 --- 
A3 : => !c
 ***** 
 ***** 
A12 : A2, A3 -> !d
 --- 
A2 : => b
A3 : => !c
 ***** 
 ***** 
A13 : A3, A4 -> !b
 --- 
A4 : => d
A3 : => !c
 ***** 
 ***** 
A14 : A3, A7 -> !b
 --- 
A7 : A3 -> d
A3 : => !c
 ***** 
 ***** 
A15 : A5 -> c
 --- 
A1 : -> a
A5 : A1 => !d
 ***** 
 ***** 
A16 : A15 => e
 --- 
A15 : A5 -> c
A1 : -> a
A5 : A1 => !d
 ***** 
 ***** 
A17 : A12 -> c
 --- 
A12 : A2, A3 -> !d
A2 : => b
A3 : => !c
 ***** 
 ***** 
A18 : A17 => e
 --- 
A12 : A2,

# 2/ Generate attacks

## 2.1/ Undercuts

In [24]:
def non_contredit(arguments):
    non_contre = []
    for i, arg1 in enumerate(arguments):
        contre_trouve = False
        for j, arg2 in enumerate(arguments):
            if i != j: 
                conclusion_neg_arg1 = arg1.top_rule.conclusion.copy()
                conclusion_neg_arg1.negate()
                if conclusion_neg_arg1 == arg2.top_rule.conclusion:
                    contre_trouve = True
                    break
        if not contre_trouve:
            non_contre.append(arg1)
    return non_contre

def is_defaisible(non_contre):
    defaisible = []
    for arg in non_contre:
        if arg.top_rule.is_defeasible:
            all_def = True
            for sub_arg in arg.sub_arguments:
                if not sub_arg.top_rule.is_defeasible:
                    all_def = False
                    break
            if all_def == True:
                defaisible.append(arg)
    return defaisible


def undercut(arguments):
    non_contre = non_contredit(arguments)
    defaisibles = is_defaisible(non_contre)
    under = []
    for defaisible in defaisibles:
        
        for arg in arguments:
            
            all_def_rules = arg.all_defeasible_rules()
           
            
            for rule in all_def_rules:
                defa = defaisible.top_rule.conclusion
                defa_neg = defa.copy()
                defa_neg.negate()
                if (rule.reference == defa) or (rule.reference == defa_neg) :
                    under.append((defaisible,arg))
    
    return under

under = undercut(arguments)

for und in under:
    print(f"({und[0].name}, {und[1].name})")

(A11, A5)
(A11, A15)
(A11, A16)


## 2.2/ Rebuts

In [25]:
def already_in_list(reb, list):
    already = False
    for arg in list:
        if reb == arg:
            already = True
            break
    return already

def tuple_already_in_list(tup, lst):
    already = False
    for item in lst:
        if item == tup:
            already = True
            break
    return already

def contredit(arguments):
    contre = []
    for i, arg1 in enumerate(arguments):
        for j, arg2 in enumerate(arguments):
            if i != j:
                conclusion_neg_arg1 = arg1.top_rule.conclusion.copy()
                conclusion_neg_arg1.negate()
                if conclusion_neg_arg1 == arg2.top_rule.conclusion:
                    if already_in_list(arg1, contre) == False:
                        contre.append(arg1)
    return contre

def rebuts_rec(cnt, arg, arguments, reb):
    for arg2 in arguments:
        for sub_arg in arg2.sub_arguments:
            if arg == sub_arg:
                pair = (cnt, arg2)
                if not tuple_already_in_list(pair, reb):
                    reb.append(pair)
                    rebuts_rec(cnt, arg2, arguments, reb)
    return reb
    

def rebuts(arguments):
    contre = contredit(arguments)
    reb = []
    for cnt in contre:
        cnt_neg = cnt.top_rule.conclusion.copy()
        cnt_neg.negate()
        for arg in arguments:
            if arg.top_rule.conclusion == cnt_neg:
                reb.append((cnt, arg))
                rebuts_rec(cnt, arg, arguments, reb)
                
                
    return reb

        

contre = rebuts(arguments)
for pair in contre:
    print(f"({pair[0].name}, {pair[1].name})")
    
print(len(contre))

(A2, A13)
(A2, A14)
(A3, A6)
(A3, A9)
(A3, A8)
(A3, A10)
(A3, A15)
(A3, A16)
(A3, A17)
(A3, A18)
(A4, A5)
(A4, A15)
(A4, A16)
(A4, A12)
(A4, A17)
(A4, A18)
(A5, A4)
(A5, A6)
(A5, A9)
(A5, A13)
(A5, A7)
(A5, A8)
(A5, A10)
(A5, A14)
(A6, A3)
(A6, A7)
(A6, A8)
(A6, A10)
(A6, A14)
(A6, A11)
(A6, A12)
(A6, A17)
(A6, A18)
(A6, A13)
(A7, A5)
(A7, A15)
(A7, A16)
(A7, A12)
(A7, A17)
(A7, A18)
(A8, A3)
(A8, A7)
(A8, A8)
(A8, A10)
(A8, A14)
(A8, A11)
(A8, A12)
(A8, A17)
(A8, A18)
(A8, A13)
(A12, A4)
(A12, A6)
(A12, A9)
(A12, A13)
(A12, A7)
(A12, A8)
(A12, A10)
(A12, A14)
(A13, A2)
(A13, A6)
(A13, A9)
(A13, A8)
(A13, A10)
(A13, A12)
(A13, A17)
(A13, A18)
(A14, A2)
(A14, A6)
(A14, A9)
(A14, A8)
(A14, A10)
(A14, A12)
(A14, A17)
(A14, A18)
(A15, A3)
(A15, A7)
(A15, A8)
(A15, A10)
(A15, A14)
(A15, A11)
(A15, A12)
(A15, A17)
(A15, A18)
(A15, A13)
(A17, A3)
(A17, A7)
(A17, A8)
(A17, A10)
(A17, A14)
(A17, A11)
(A17, A12)
(A17, A17)
(A17, A18)
(A17, A13)
94


# 3/ Generate defeats


In [35]:
def generate_defeats_democratic(preferences):
    defeats = []
    for rule, preferred_rules in preferences.items():
        for preferred_rule in preferred_rules:
            defeats.append((preferred_rule, rule))
    return defeats

def generate_defeats_weakest_link(preferences):
    defeats = []
    for rule, preferred_rules in preferences.items():
        for preferred_rule in preferred_rules:
            defeats.append((rule, preferred_rule))
    return defeats

# Example preferences
preferences = {
    "r4": ["r2", "r7", "r8", "r9"],
    "r6": ["r2", "r7", "r8", "r9"]
}
defeats = []
# Generate defeats using democratic principle
defeats_democratic = generate_defeats_democratic(preferences)
print("Defeats using democratic principle:")
defeats.extend(defeats_democratic)
print(len(defeats))

#for defeat in defeats_democratic:
 #   print(defeat)

# Generate defeats using weakest-link principle
defeats_weakest_link = generate_defeats_weakest_link(preferences)
print("\nDefeats using weakest-link principle:")
defeats.extend(defeats_weakest_link)
print(len(defeats_weakest_link))



Defeats using democratic principle:
8

Defeats using weakest-link principle:
8
